<a href="https://colab.research.google.com/github/TasnimTamanna02/Building-ReAct-Agent/blob/main/ReAct_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 %pip install groq #Groq is a site that allows the use of open-source LLM models through API key for free

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/Assingments/AI Practice/ReAct Agent/api_key.txt', 'r') as f:
         api_key = f.read().strip()
api_key

'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'

In [ ]:
import os
os.environ['GROQ_API_KEY']= api_key

## Sample Test

In [ ]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
#this client object is the primary communication channel to interact with Groq API
chat_completion = client.chat.completions.create( #This line uses the client object to send a request to the language model for generating text. It's like saying, "Hey language model, please create a completion for this chat."
    messages=[ #to give context to the model
        {
            "role": "user", #msg coming from user
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile", #the model used for this program. You can select as you want from Groq's website
)

print(chat_completion.choices[0].message.content)
#choices: The response object contains a list of possible responses. Typically, there's only one choice, so we access it using [0].
#message: This object contains details about the LLM's response, including the content.
#content: This extracts the actual textual response generated by the LLM.

## Building Agent

In [ ]:
class Agent:
  def __init__(self, client, system):
    self.client=client # Holds the client object to communicate with the language model.
    self.system=system # Stores the system prompt that defines the model’s behavior (used only if provided).
    self.messages= [] # A list that stores the full conversation history with roles and messages.
    if self.system is not None: #Checks if a system prompt was provided
      self.messages.append({"role":"system", "content": self.system}) #If there is a system prompt, it's added to the messages list

  def __call__(self, message=""): #makes an object callable like a function
    if message: #True for non-empty, False for empty
      self.messages.append({"role":"user", "content": message}) # Adds the user’s new message to the message history
    result = self.execute()  # Calls the execute method to get model’s response
    self.messages.append({"role":"assistant", "content": result})  # Stores the model's reply in the message history
    return result

  def execute(self): #Accessing the model
    completion = self.client.chat.completions.create(
        messages= self.messages, # Sends full message history to the model
        model="llama3-70b-8192",
    )
    return completion.choices[0].message.content #Only returns the text part (the actual response) from the model's reply
    #completion contains everything the model returns
    #choice is a list of possible response, by [0] we take the first one
    #message contains 'role' and 'content'
    #Content is the actual text response we need

# Roles in the messages list:
# "system" – Sets the assistant’s behavior (e.g., tone, purpose) before any conversation starts.
# "user" – Represents messages coming from the user (you or the caller of the agent).
# "assistant" – Represents responses generated by the AI model.

In [ ]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
Returns weight of the planet in kg


Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1.1944e+25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1.1944e+25.

Now it's your turn:
""".strip()

In [ ]:
#tools
def calculate(operation):
  return eval(operation) #evaluates the string as a Python expression

# WARNING: Using eval() is risky because it can execute arbitrary code.
# Never use it with untrusted input, as it can lead to security vulnerabilities.
# Safe only for controlled or internal testing environments.


def get_planet_mass(planet) -> float:
  match planet.lower():
    case "earth":
      return 5.972e24
    case "mars":
      return 6.39e23
    case "jupiter":
      return 1.898e27
    case "saturn":
      return 5.683e26
    case "uranus":
      return 8.681e25
    case "mercury":
      return 3.301e23
    case "venus":
      return 4.867e24
    case "neptune":
      return 1.024e26
    case "pluto":
      return 1.27e22
    case _:
      return 0.0

## Running Agent (Manually)

In [ ]:
Neil_from_Nasa = Agent(client, prompt) #Neil_from_Nasa is the object and prompt is the system message

In [ ]:
result = Neil_from_Nasa("What is the mass of Mars times 5?")
print(result)

Thought: I need to find the mass of Mars.


In [ ]:
Neil_from_Nasa.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nReturns weight of the planet in kg\n\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1.1944e+25\n\nIf you have the a

In [ ]:
result = Neil_from_Nasa()
print(result)

Action: get_planet_mass: Mars
PAUSE


In [ ]:
observations = get_planet_mass("Mars")
print(observations)

6.39e+23


In [ ]:
next_prompt = f"Observation: {observations}"
result = Neil_from_Nasa(next_prompt)
print(result)

Thought: I need to multiply this by 5.


In [ ]:
Neil_from_Nasa.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nReturns weight of the planet in kg\n\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1.1944e+25\n\nIf you have the a

In [ ]:
result = Neil_from_Nasa()
print(result)

In [ ]:
observations = calculate("5.683e26 * 5")
print(observations)

2.8415e+27


In [ ]:
next_prompt = f"Observation: {observations}"
result = Neil_from_Nasa(next_prompt)
print(result)

Answer: The mass of Mars times 5 is 2.8415e+27.


In [ ]:
Neil_from_Nasa.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nReturns weight of the planet in kg\n\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1.1944e+25\n\nIf you have the a

## Running Agent (Autonomously)

In [ ]:
import re #regular expression

def agent_loop(max_iterations, system, query):
  agent = Agent(client, prompt)
  tools = {'calculate', 'get_planet_mass'}
  next_prompt = query
  i = 0
  tool_functions = {
    'calculate': calculate,
    'get_planet_mass': get_planet_mass,} #dictionary mapping tool names to the actual Python functions that run them.

  while i<max_iterations:  # Loops until the maximum number of allowed iterations
    i=1 #should be i+=1. but that somehow wrecks the process,the output either doesn't come or comes faulty. Couldn't figure out why.
    result = agent(next_prompt)
    print(result)

    if "PAUSE" in result and "Action" in result:  #Checks if the agent wants to pause and has requested an action.
      action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
                            # re.findall(): uses Python's regex module to find all matches of the pattern in the text
                            # r"...": raw string notation so backslashes are treated literally
                            # ([a-z_]+): captures the tool name (letters or underscores)
                            # (.+): captures the tool argument (everything after the second colon)
                            # re.IGNORECASE: makes matching case-insensitive (e.g., matches "Action" or "action")
      chosen_tool = action[0][0]
      arg = action[0][1]

      if chosen_tool in tools:
        result_tool = tool_functions[chosen_tool](arg) #In the tutorial: result_tool = eval(f"{chosen_tool}('{arg}')") but this isn't working for me
        next_prompt = f"Observation: {result_tool}"

      else:
        next_prompt = "Observation Tool not found."

      print(next_prompt)
      continue
    if "Answer" in result:
      break

In [ ]:
agent_loop(7, prompt, "What is the mass of Mars plus the mass of Earth, and all of it times 5?")

Thought: I need to find the masses of Mars and Earth, then add them, and finally multiply by 5
Thought: I need to find the masses of Mars and Earth
Action: get_planet_mass: Mars
PAUSE
Observation: 6.39e+23
Thought: I have the mass of Mars, now I need to get the mass of Earth
Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: I have the masses of Mars and Earth, now I need to add them
Action: calculate: 6.39e+23 + 5.972e+24
PAUSE
Observation: 6.611000000000001e+24
Thought: I have the sum of the masses, now I need to multiply it by 5
Action: calculate: 6.611000000000001e+24 * 5
PAUSE
Observation: 3.3055e+25
Thought: I have the final result
Answer: The mass of Mars plus the mass of Earth, and all of it times 5 is 3.3055e+25.


## Notes

### Agent Loop Explanation
Thought → Action → Observation → Answer cycle


1.   Agent thinks and decides what action to take based on the prompt and previous messages.
2.   loop (our code) detects this action using regex, extracts the tool name and its argument, and runs the actual tool function (like get_planet_mass("Earth"))
3. returns the Observation (the tool's output) back to the Agent as the next prompt.
4. repeats until the Agent outputs an Answer






In [ ]:
# OG!Prompt- [taken from Simon Willison TILs blog post]
# prompt = """
# You run in a loop of Thought, Action, PAUSE, Observation.
# At the end of the loop you output an Answer
# Use Thought to describe your thoughts about the question you have been asked.
# Use Action to run one of the actions available to you - then return PAUSE.
# Observation will be the result of running those actions.

# Your available actions are:

# calculate:
# e.g. calculate: 4 * 7 / 3
# Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

# wikipedia:
# e.g. wikipedia: Django
# Returns a summary from searching Wikipedia

# simon_blog_search:
# e.g. simon_blog_search: Django
# Search Simon's blog for that term

# Always look things up on Wikipedia if you have the opportunity to do so.

# Example session:

# Question: What is the capital of France?
# Thought: I should look up France on Wikipedia
# Action: wikipedia: France
# PAUSE

# You will be called again with this:

# Observation: France is a country. The capital is Paris.

# You then output:

# Answer: The capital of France is Paris
# """.strip()